In [ ]:
#Contraparte e implementación con librerías Tradicionales con más datos
#Pandas, Scikit-learn, Matplotlib

In [1]:
#Importat librerias a utilizar
import pandas as pd 
import json
from pandas.io.json import json_normalize

In [ ]:
##########Carga con Pandas###########

In [2]:
#exportación y lectura con pandas
import time

inicio = time.time()

pd_Pandas = pd.read_csv('df_bigdata.csv')
fin = time.time()

calcultedPD = fin-inicio
print("Cálculo de extracción y lectura con Pandas = {}".format(calcultedPD))

Cálculo de extracción y lectura con Pandas = 10.276310443878174


In [3]:
print('Descripción de la extracción y lectura del archivo:')
pd_Pandas

Descripción de la extracción y lectura del archivo:


,u,n,o
0,/edx/var/log/tracking/tracking.log-20171110-15...,NaN,NaN
1,/edx/var/log/tracking/tracking.log-20171110-15...,NaN,NaN
2,/edx/var/log/tracking/tracking.log-20171110-15...,NaN,NaN
3,/edx/var/log/tracking/tracking.log-20171110-15...,NaN,NaN
4,/edx/var/log/tracking/tracking.log-20171110-15...,NaN,NaN
...,...,...,...
399995,/edx/var/log/tracking/tracking.log-20171220-15...,NaN,NaN
399996,/edx/var/log/tracking/tracking.log-20171220-15...,NaN,NaN
399997,/edx/var/log/tracking/tracking.log-20171220-15...,NaN,NaN
399998,/edx/var/log/tracking/tracking.log-20171220-15...,NaN,NaN


In [ ]:
########Transformación con Pandas################

In [6]:
#convertir en lista y despues en json
import time

startSecuenciaCPU = time.time()

data = list(pd_Pandas['u'])
da = []
for i in range(0,len(data)):
    d = data[i].split('.gz:')
    da.append(json.loads((d[1])))
    
#DataFrame Panda
new_data = pd.DataFrame(da)

#solo conservo los datos de la columna username
#mayor a 0
snNull = new_data[new_data.username.map(len) > 0]

#eliminar username dicc y opencampus
snDicc = snNull[snNull.username != 'dicc']
snCampus = snDicc[snDicc.username != 'opencampus']

#eliminar filas con los datos perdidos
snCampus.dropna(axis=0, how='any')

#Eliminar columnas innecesarias no tiene relevancia
snCampus.drop(['accept_language'],axis=1, inplace=True)
snCampus.drop(['name'], axis=1, inplace=True)
snCampus.drop(['agent'], axis=1, inplace=True)
snCampus.drop(['page'], axis=1, inplace=True)
snCampus.drop(['session'], axis=1, inplace=True)
snCampus.drop(['event'], axis=1, inplace=True)
snCampus.drop(['event_type'], axis=1, inplace=True)

#eliminar filas con datos perdidos
snCampus.dropna(axis=0, how='any')

dataLimpia = snCampus

#Dividir toda la columna context
contextData = dataLimpia['context'].values.tolist()
new_contextData = pd.DataFrame(contextData, columns= ['course_user_tags',
                                                      'course_id', 'path',
                                                      'org_id', 'user_id'])
generalData = pd.merge(dataLimpia.reset_index(),
                       new_contextData.reset_index(),
                       left_index=True, right_index=True)
generalData = generalData.drop(['context'], axis=1)
generalData = generalData.drop(['index_y'], axis=1)
generalData = generalData.drop(['course_user_tags'], axis=1)
generalData = generalData.drop(['course_id'], axis=1)
generalData = generalData.drop(['path'], axis=1)

#eliminar filas con datos perdidos
generalData.dropna(axis=0, how='any')

#soltar duplicados
snCampus.sort_values("username") 
snCampus.drop_duplicates(subset='username',
                         keep=False, inplace=False)

#dividir columna time
df_picru = pd.DataFrame(generalData['time'])
datenew = df_picru.apply(lambda x: pd.to_datetime(x,errors='coerce',
                                                  format = '%Y-%m-%d'))
#Asignar a cada columna representacipon del tipo
#de dato
datenew['Day'] = datenew['time'].dt.day
datenew['Month'] = datenew['time'].dt.month
datenew['Year'] = datenew['time'].dt.year
datenew['Hour'] = datenew['time'].dt.time

#Eliminar columnas
datenew = datenew.drop(['time'], axis=1)

#Unir todo el dataframe time al general
generalData = pd.merge(generalData.reset_index(),
                       datenew.reset_index(),
                       left_index=True, right_index=True)
generalData = generalData.drop(['time','index_x',
                                'index_y'], axis=1)

#eliminar filas con datos perdidos
generalData.dropna(axis=0, how='any')

#resetear index
generalData = generalData.reset_index()
generalData = generalData.drop(['index'], axis=1)

endSecuenciaCPU = time.time()
cpuPandas = endSecuenciaCPU - startSecuenciaCPU

print('Cálculo de transformación cpu-Pandas:', cpuPandas)

/opt/conda-environments/rapids-stable/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Cálculo de transformación cpu-Pandas: 17.070501565933228


In [11]:
#Información de toda la data con Pandas
print('Nomnres de las columnas:',generalData.columns)
print('Dimension del DataFrame:',generalData.shape)
print('Tipos de datos de cada columna', generalData.dtypes)
print('Información DATA general con Pandas:')
generalData.head(2)

Nomnres de las columnas: Index(['username', 'event_source', 'host', 'referer', 'ip', 'org_id',
       'user_id', 'Day', 'Month', 'Year', 'Hour'],
      dtype='object')
Dimension del DataFrame: (393734, 11)
Tipos de datos de cada columna username        object
event_source    object
host            object
referer         object
ip              object
org_id          object
user_id          int64
Day              int64
Month            int64
Year             int64
Hour            object
dtype: object
Información DATA general con Pandas:


,username,event_source,host,referer,ip,org_id,user_id,Day,Month,Year,Hour
0,FrankVac,server,opencampus.utpl.edu.ec,http://opencampus.utpl.edu.ec/courses,200.105.237.152,UTPL,10838,10,11,2017,19:43:50.202806
1,FrankVac,server,opencampus.utpl.edu.ec,http://opencampus.utpl.edu.ec/courses/course-v...,200.105.237.152,,10838,10,11,2017,19:43:50.759372


In [ ]:
######Carga con Pandas############

In [14]:
#Cargar data entendible en un nuevo csv
start = time.time()

da_data = 'data_mejoraPandas_grande.csv'
generalData.to_csv(da_data)

end = time.time()
calculated = end - start

print("Cálculo de carga cpu-Pandas = {}".format(calculated))

Cálculo de carga cpu-Pandas = 3.5496439933776855


In [15]:
#Extracción y lectura de toda la data transformada
nueva_data = pd.read_csv('data_mejoraPandas_grande.csv')
nueva_data = nueva_data.drop(['Unnamed: 0'],axis=1)

#información de la nueva data
print('Tipos de datos de cada columna', nueva_data.dtypes)
print('Información Data')
nueva_data.head(2)

Tipos de datos de cada columna username        object
event_source    object
host            object
referer         object
ip              object
org_id          object
user_id          int64
Day              int64
Month            int64
Year             int64
Hour            object
dtype: object
Información Data


,username,event_source,host,referer,ip,org_id,user_id,Day,Month,Year,Hour
0,FrankVac,server,opencampus.utpl.edu.ec,http://opencampus.utpl.edu.ec/courses,200.105.237.152,UTPL,10838,10,11,2017,19:43:50.202806
1,FrankVac,server,opencampus.utpl.edu.ec,http://opencampus.utpl.edu.ec/courses/course-v...,200.105.237.152,NaN,10838,10,11,2017,19:43:50.759372


In [ ]:
#####Proceso Machine learning########

In [16]:
#Fase Dividir dataset con sklearn
from sklearn.model_selection import train_test_split

#Asignar dataset general a nuevas variables
pd_dataX = nueva_data
pd_dataY = nueva_data

#Dividir 4 datasets en train y test
#datatrain 80% y datatest 20%
X_train, X_test, y_train, y_test = train_test_split(pd_dataX,
                                                    pd_dataY, train_size=0.8)

#información
print(f'Dataset Original: {pd_dataX.shape[0], pd_dataY.shape[0]} elementos')
print(f'Data X_train: {X_train.shape[0]} elementos')
print(f'Data X_test: {X_test.shape[0]} elementos')
print(f'Data y_train: {y_train.shape[0]} elementos')
print(f'Data y_test: {y_test.shape[0]} elementos')

Dataset Original: (393734, 393734) elementos
Data X_train: 314987 elementos
Data X_test: 78747 elementos
Data y_train: 314987 elementos
Data y_test: 78747 elementos


In [18]:
#Fase Preprocesamiento con sklearn
import time
from sklearn import preprocessing

inicio_Pre = time.time()

le = preprocessing.LabelEncoder()

#seleccionar columnas potenciales para work train de xTrain
value_usernameX = le.fit_transform(X_train.username)
value_EventSourceX = le.fit_transform(X_train.event_source)
value_DayX = le.fit_transform(X_train.Day)

#seleccionar columnas potenciales para work test de xTest
value_usernameXT = le.fit_transform(X_test.username)
value_EventSourceXT = le.fit_transform(X_test.event_source)
value_DayXT = le.fit_transform(X_test.Day)

#seleccionar columnas potenciales para work train de yTrain
value_usernameY = le.fit_transform(y_train.username)
value_EventSourceY = le.fit_transform(y_train.event_source)
value_DayY = le.fit_transform(y_train.Day)

#seleccionar columnas potenciales para work test de yTest
value_usernameYT = le.fit_transform(y_test.username)
value_EventSourceYT = le.fit_transform(y_test.event_source)
value_DayYT = le.fit_transform(y_test.Day)

fin_Pre = time.time()
calculo_Pre_ML = fin_Pre - inicio_Pre

print("Cálculo fase preprocesamiento con sklearn = {}".format(calculo_Pre_ML))

Cálculo fase preprocesamiento con sklearn = 0.2811887264251709


In [19]:
#Algoritmo LinearRegression
import time
import numpy as np
from sklearn.linear_model import LinearRegression

inicio_RL = time.time()

#Creación de un nuevo dataframe Train
dataTrain = pd.DataFrame()
dataTrain['username'] = np.array(value_usernameX,
                                 dtype=np.float32)
dataTrain['day'] = np.array(value_DayX,
                            dtype=np.float32)

#Creación de un nuevo dataframe Test
new_value = np.array(value_usernameY,
                     dtype=np.float32)
dataY = pd.Series(new_value)

#Modelo de entrenamiento usando el algoritmo
lr_reg = LinearRegression(fit_intercept= True, normalize = False)
reg = lr_reg.fit(dataTrain, dataY)

fin_RL = time.time()
calculo_Algorit_ML = fin_RL - inicio_RL

print("Algoritmo Regresión Lineal")
print("Coeficientes:")
print(reg.coef_)
print("Intercepción:")
print(reg.intercept_)
print('Predicción del modelo:')
print(lr_reg.predict(dataTrain))
print('')

print("Cálculo fase de entrenamiento algoritmo Machine Learning Regresión Lineal con sklearn = {}".format(calculo_Algorit_ML))

Algoritmo Regresión Lineal
Coeficientes:
[1.0000066e+00 5.1090791e-07]
Intercepción:
-0.0015563965
Predicción del modelo:
[220.9999  492.00168 220.99991 ... 220.99991 220.99991 220.99991]

Cálculo fase de entrenamiento algoritmo Machine Learning Regresión Lineal con sklearn = 0.09604215621948242


In [ ]:
#Algoritmo Regresión Logística de sklearn
import time
import numpy as np
from sklearn.linear_model import LogisticRegression

inicio_RL = time.time()

#Creación de un nuevo dataframe Train 
dataTrain = pd.DataFrame()
dataTrain['username'] = np.array(value_usernameX, dtype=np.float32)
dataTrain['day'] = np.asarray(value_DayX, dtype=np.float32)

#Creación de un nuevo dataframe Test
new_value = np.array(value_usernameY, dtype=np.float32)
dataTest = pd.Series(new_value)

#Modelo de entrenamiento usando el algoritmo
reg = LogisticRegression(fit_intercept = True,
                         max_iter = 100)
reg.fit(dataTrain, dataTest)

fin_RL = time.time()
calculo_RL = fin_RL - inicio_RL

print("Coefficients:")
print(reg.coef_)
print("Intercept:")
print(reg.intercept_)
print('Predicción del modelo:')
print(reg.predict(dataTrain))
print('')

print("Cálculo del algoritmo Regresión Logistica con sklearn = {}".format(calculo_RL))

In [7]:
#El kernel de BlanzingSQL para entrenar el algoritmo Regresión Logística no pudo ser soportado
#por ende no se muestra resultado alguno del entrenamiento